In [1]:
# test if the tf-idf feature can show the similarity between bugs. This file is not used in the final system.

import MySQLdb
import pandas
import re
import itertools
import numpy as np
import nltk
#LCAL_DATABASE_HOST = "10.117.8.41"
#LOCAL_DATABASE_PORT = 3306
#LOCAL_DATABASE_USER = "root"
#LOCAL_DATABASE_PW = "vmware"
#LOCAL_DATABASE_DATABASE = "bugdata"
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur =conn.cursor()

#sql = '''SELECT bug_id 
#FROM bugs
#WHERE assigned_to in (
#SELECT userid 
#FROM profiles 
#WHERE login_name in ('hfu','letian','vbhakta','weili','nmukuri','zhoum','hxie','shiyaoy',
#'shanpeic','souravk','vaibhavk','fangchiw','gengshengl'))
#LIMIT 0, 1000'''

sql = '''SELECT bug_id 
FROM bugs,profiles 
WHERE bugs.assigned_to=profiles.userid and profiles.login_name in ('hfu','letian','vbhakta','weili','nmukuri','zhoum','hxie','shiyaoy','shanpeic','souravk','vaibhavk','fangchiw','gengshengl')
LIMIT 0, 1000'''

cur.execute(sql)
bug_id = cur.fetchall()
bug_id = [str(item[0]) for item in bug_id]
print len(bug_id)


1000


In [2]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur =conn.cursor()
sql_sd = '''SELECT bug_id, short_desc
FROM bugs
WHERE bug_id IN (%s)
ORDER BY bug_id''' %  ','.join(bug_id)

df_sd = pandas.io.sql.read_sql(sql_sd, conn)

print df_sd.dtypes

bug_id         int64
short_desc    object
dtype: object


In [3]:
sql_ld = '''SELECT bug_id, group_concat('\n', longdescs.thetext) description
FROM longdescs
WHERE bug_id IN (%s)
GROUP BY bug_id
ORDER BY bug_id''' % ','.join(bug_id)

df_ld = pandas.io.sql.read_sql(sql_ld, conn)

print df_ld.dtypes

bug_id          int64
description    object
dtype: object


/root/.virtualenvs/dup_bug_retrieval/lib/python2.7/site-packages/pandas/io/sql.py:1462: Warning: 785 line(s) were cut by GROUP_CONCAT()
  cur.execute(*args)


In [4]:
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup

def discription_to_wordlist( raw_description ):
    # Function to convert a raw bug description to a list of words
    #  Remove markers
    text = BeautifulSoup(raw_description).get_text() 
    #
    # Remove non-letters        
    # letters_only = re.sub("[^a-zA-Z_/\-\.]", " ", description_text)
    letters_only = re.sub("[^a-zA-Z\.]", " ", text)
    letters_only = re.sub("\.(?!((c|h|cpp|py)\s+$))", " ", letters_only)
    #
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    meaningful_words = [w for w in words if not w in stops]   
    #
    # Stemming
    porter = PorterStemmer()
    stemmed_meaningful_words = [porter.stem_word(w) for w in meaningful_words]
    
    return stemmed_meaningful_words

In [5]:
def discription_to_string( raw_description ):
    
    return " ".join(discription_to_wordlist(raw_description))

In [6]:
from nltk.stem.snowball import SnowballStemmer

def tokenize_and_stem( raw_description ):
    text = BeautifulSoup(raw_description).get_text()
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    stopwords = set(nltk.corpus.stopwords.words('english'))
    meaningful_tokens = [token for token in filtered_tokens if not token in stopwords]
    snowball = SnowballStemmer("english")
    stems = [snowball.stem(t) for t in filtered_tokens]
    return stems

In [7]:
def tokenizestem_to_string( raw_description ):
    
    return " ".join(tokenize_and_stem(raw_description))

In [11]:
df = pandas.merge(df_sd, df_ld, on='bug_id')
texts = list((df['short_desc'] +' '+ df['description']).map(tokenizestem_to_string))

In [12]:
# print list(df['string'])

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",ngram_range=(1, 2), tokenizer = None, preprocessor = None,
                             stop_words = None, lowercase = False, 
                             max_df = 0.9, min_df = 10, max_features = 100000)

desc_vectors = vectorizer.fit_transform(texts)

In [27]:
print vectorizer.vocabulary_

{u'api affect': 136, u'take this': 1830, u'or review': 1375, u'wang': 2269, u'want': 2271, u'number vmware': 1321, u'wrong': 2361, u'chang what': 372, u'are not': 158, u'some': 1755, u'fix': 734, u'fil': 716, u'timeout': 1982, u'adapt': 70, u'vmci': 2224, u'to find': 2008, u'unit test': 2110, u'vmware com': 2244, u'provid the': 1527, u'task': 1834, u'com ob': 417, u'is need': 1035, u'at least': 197, u'patch': 1420, u'power off': 1489, u'assum': 194, u'result in': 1633, u'much': 1248, u'for pr': 764, u'rdm disk': 1552, u'had': 845, u'has': 852, u'hat': 856, u'pbms': 1429, u'log for': 1151, u'main': 1175, u'usb arbitr': 2139, u'notic': 1310, u'osp': 1385, u'vm on': 2214, u'link http': 1130, u'exact': 680, u'modif': 1225, u'address': 75, u'more detail': 1233, u'queue': 1543, u'instal': 991, u'no recent': 1285, u'the new': 1910, u'sr vmware': 1782, u'vmfs volumes': 2229, u'disconnect': 571, u'the follow': 1898, u'onc': 1354, u'the vmware': 1935, u'vc version': 2168, u'root password': 1657,

In [28]:
print len(vectorizer.vocabulary_)

2391


In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidf_vectors = tfidf_transformer.fit_transform(desc_vectors)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf_vectors.toarray())
print similarity.shape
print similarity

(1000, 1000)
[[ 1.          0.01546298  0.10670418 ...,  0.09659317  0.06715389
   0.16303238]
 [ 0.01546298  1.          0.12377881 ...,  0.08992518  0.04976035
   0.03547346]
 [ 0.10670418  0.12377881  1.         ...,  0.11374965  0.05719224
   0.13497978]
 ..., 
 [ 0.09659317  0.08992518  0.11374965 ...,  1.          0.17393915
   0.09978197]
 [ 0.06715389  0.04976035  0.05719224 ...,  0.17393915  1.          0.06607398]
 [ 0.16303238  0.03547346  0.13497978 ...,  0.09978197  0.06607398  1.        ]]


In [31]:
import numpy as np
sim_exc_self = similarity - np.eye(similarity.shape[0],similarity.shape[1])
max_row = sim_exc_self.argmax(1)
lst = [array[max_row[i]] for i, array in enumerate(sim_exc_self)]
max_column = lst.index(max(lst))
print df['bug_id'][max_row[max_column]]
print df['bug_id'][max_column]
print sim_exc_self[max_column,max_row[max_column]]
for i, item in enumerate(max_row):
    print df['bug_id'][i], df['bug_id'][item],sim_exc_self[i,item]

1134233
1134070
1.0
245670 874567 0.266862947237
258045 504958 0.513555715459
326880 983072 0.320983228718
365196 1005553 0.302337025229
400528 1160570 0.39446958717
400907 834752 0.355438292934
404886 586569 0.477665654961
404925 555018 0.43285688595
406014 498046 0.413495098438
406544 873438 0.528463240893
407331 586083 0.433254827194
408331 1320797 0.336493908906
411852 447779 0.480624811064
414335 461058 0.41683865927
419957 641931 0.421732117911
420631 498650 0.438183135439
422171 422802 0.335144375339
422802 1382777 0.577999372846
425313 654800 0.411980185949
444989 519565 0.99819367211
446209 404886 0.346660189056
447779 411852 0.480624811064
447883 651370 0.311477154589
450273 482099 0.543541241057
451096 763714 0.336596719124
452896 461705 0.49194637953
453982 406544 0.426557366894
456095 537741 0.423392307711
457822 1202489 0.299932173537
458314 607909 0.847542910848
460416 414335 0.392775093696
460493 922139 0.364344463943
460795 601597 0.369776029729
461058 467772 0.6313774